In [1]:
import os, sys
import glob
import numpy as np
sys.path.append(os.path.join(os.getenv('LAMBO_HOME'), 'lambo/scripts/'))

from hsc_gaap.gaap import findReducedPatches
from hsc_gaap.check_gaap_run import checkRun
%load_ext autoreload
%autoreload 2


Overriding default configuration file with /projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/dustmaps_cachedata/g41a3ec361e+62d12e78cb/config/.dustmapsrc


In [8]:
tracts = [8282, 8283, 8284, 8285, 8286, 9326, 9327, 9328, 9329, 8522, 8523, 8526, 8527, 8768, 8769, 9010, 9011, 9071, 9072, 9073,
          9074, 9075, 9076, 9077, 9078, 9079, 9080, 9081, 9082, 9083, 9084, 9086, 9086, 9087, 9088, 9089, 9090, 9091, 9092, 9096,
          9097, 9098, 9099, 9100, 9101, 9102, 9103, 9104, 9131, 9132, 9133, 9134, 9135, 9136, 9226, 9227, 9313, 9314, 9315, 9316, 
          9317, 9318, 9319, 9320, 9321, 9322, 9323, 9324, 9325, 9330, 9331, 9332, 9333, 9334, 9335, 9336, 9339, 9340, 9341, 9342,  
          9343, 9344, 9345, 9346, 9347, 9373, 9374, 9375, 9376, 9377, 9378, 9379, 9455, 9456, 9457, 9458, 9459, 9465, 9466, 9467,
          9468, 9469, 9470, 9471, 9556, 9557, 9558, 9559, 9560, 9561, 9562, 9563, 9564, 9565, 9568, 9586, 9587, 9594, 9813, 9836, 
          10059, 10284]

In [20]:
newtracts=[9812, 9814, 9829, 9830, 9831, 9832, 9834, 9835, 10054, 10055, 10056, 10071, 10072, 10073, 10074, 10075, 10076, 10077]

In [2]:
newertracts = [9566, 9567, 9568, 9569, 9570, 9571, 9572, 
               9573, 9574, 9575, 9576, 9577, 9578, 9579,
               9580, 9582, 9583, 9584, 9585, 9587, 
               9588, 9589, 9590, 9591, 9592, 9593, 9594, 9595, 9596]

In [5]:
print(newertracts[9:])

[9575, 9576, 9577, 9578, 9579, 9580, 9582, 9583, 9584, 9585, 9587, 9588, 9589, 9590, 9591, 9592, 9593, 9594, 9595, 9596]


In [3]:
for tract in newertracts:
    problems = checkRun(tract)

TRACT: 9566
NO PROBLEMS

TRACT: 9567
NO PROBLEMS

TRACT: 9568
NO PROBLEMS

TRACT: 9569
NO PROBLEMS

TRACT: 9570
NO PROBLEMS

TRACT: 9571
NO PROBLEMS

TRACT: 9572
NO PROBLEMS

TRACT: 9573
NO PROBLEMS

TRACT: 9574
NO PROBLEMS

TRACT: 9575
NO PROBLEMS

TRACT: 9576
NO PROBLEMS

TRACT: 9577
NO PROBLEMS

TRACT: 9578
NO PROBLEMS

TRACT: 9579
NO PROBLEMS

TRACT: 9580
NO PROBLEMS

TRACT: 9582
NO PROBLEMS

TRACT: 9583
NO PROBLEMS

TRACT: 9584
NO PROBLEMS

TRACT: 9585
NO PROBLEMS

TRACT: 9587
NO PROBLEMS

TRACT: 9588
NO PROBLEMS

TRACT: 9589
NO PROBLEMS

TRACT: 9590
NO PROBLEMS

TRACT: 9591
NO PROBLEMS

TRACT: 9592
NO PROBLEMS

TRACT: 9593
NO PROBLEMS

TRACT: 9594
NO PROBLEMS

TRACT: 9595
NO PROBLEMS

TRACT: 9596
NO PROBLEMS



In [34]:
problems = checkRun(10054)

TRACT: 10054
NO PROBLEMS



In [27]:
problems

array([ 3, 12, 20, 21, 29, 30, 39, 40, 48, 57])

In [34]:
len(problems)

0

In [35]:
print(", ".join(np.array(problems).astype(str)))

---

# Fixing blending matching

In [2]:
from hsc_gaap.run_gaap import matchBlendedness

In [3]:
blenderror ='Traceback (most recent call last):\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 198, in <module>\n    fire.Fire(runGaapRowColumn)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 141, in Fire\n    component_trace = _Fire(component, args, parsed_flag_args, context, name)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 475, in _Fire\n    component, remaining_args = _CallAndUpdateTrace(\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace\n    component = fn(*varargs, **kwargs)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 133, in runGaapRowColumn\n    matchBlendedness(tract, patch_cols, patch_rows, repo=repo)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 187, in matchBlendedness\n    abs(gaapCat["g_ap03Flux"] - blendCat["g_apertureflux10_S20A"][ind]) < 10)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/table/table.py", line 1872, in __getitem__\n    return self.columns[item]\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/table/table.py", line 246, in __getitem__\n    return OrderedDict.__getitem__(self, item)\nKeyError: \'g_ap03Flux\''
repo = "/scratch/gpfs/am2907/Merian/gaap/"

In [13]:
def fixBlendMatching(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/', blenderror=blenderror):
    
    # Find patches that didn't match
    print(f'TRACT: {tract}')
    problem_patches=[]

    patches = findReducedPatches(tract)
    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]
    
    if len(logs) != len(patches):
        print("MISSING PATCHES")

    error3 = "Matched GAaP table with S20A blendedness"
    for i, log in enumerate(logs):
        logfile = open(log, "r").read()

        if error3 not in logfile:
            problem_patches.append(logs_patches[i])

    for patch in problem_patches:
        print(f'PATCH: {patch}')
        log = logs[logs_patches==patch][0]
        logfile = open(log, "r").read()
        message = matchBlendedness(tract, [patch%9], [patch//9], repo=repo, matchdist=0.5, matchmag=10)
        if type(message) is not str:
            continue
        newlog_cntnt = logfile.split(blenderror)[0] + f"\n {message} \n" + logfile.split(blenderror)[1]
        newlog = open(log, "w")
        newlog.write(newlog_cntnt)
        newlog.close()

In [5]:
blenderror='Traceback (most recent call last):\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 202, in <module>\n    fire.Fire(runGaapRowColumn)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 141, in Fire\n    component_trace = _Fire(component, args, parsed_flag_args, context, name)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 475, in _Fire\n    component, remaining_args = _CallAndUpdateTrace(\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace\n    component = fn(*varargs, **kwargs)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 134, in runGaapRowColumn\n    matchBlendedness(tract, patch_cols, patch_rows, repo=repo)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 185, in matchBlendedness\n    ind, dist, _ = _gaap.match_to_catalog_sky(_blend)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/coordinates/sky_coordinate.py", line 1386, in match_to_catalog_sky\n    res = match_coordinates_sky(self, catalogcoord,\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/coordinates/matching.py", line 143, in match_coordinates_sky\n    raise ValueError(\'The catalog for coordinate matching cannot be a \'\nValueError: The catalog for coordinate matching cannot be a scalar or length-0.'

In [129]:
tract=8527

In [12]:
fixBlendMatching(8527, blenderror=blenderror)

TRACT: 8527
PATCH: 75
Blend catalog empty - cannot match!


---

In [38]:
tract = 9574
repo = '/scratch/gpfs/am2907/Merian/gaap/'

In [39]:

patches = findReducedPatches(tract)
dir = os.path.join(repo, f"log/{tract}/")
logs = np.array(glob.glob(dir + "*.o"))
logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]

In [40]:
error = 'Traceback (most recent call last):\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 210, in <module>\n    fire.Fire(runGaapRowColumn)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 141, in Fire\n    component_trace = _Fire(component, args, parsed_flag_args, context, name)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 475, in _Fire\n    component, remaining_args = _CallAndUpdateTrace(\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace\n    component = fn(*varargs, **kwargs)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 139, in runGaapRowColumn\n    compileCatalogs(tract, repo= "/scratch/gpfs/am2907/Merian/gaap/", alltracts=False, rewrite=True)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/compile_catalogs.py", line 20, in compileCatalogs\n    for tract in tracts:\nTypeError: \'int\' object is not iterable'

In [41]:
for patch in logs_patches:
    log = logs[logs_patches==patch][0]
    logfile = open(log, "r").read()
    if error in logfile:
        print(f'PATCH: {patch}')
        newlog_cntnt = logfile.split(error)[0] + logfile.split(error)[1]
        newlog = open(log, "w")
        newlog.write(newlog_cntnt)
        newlog.close()

PATCH: 0
PATCH: 1
PATCH: 2
PATCH: 3
PATCH: 4
PATCH: 5
PATCH: 6
PATCH: 7
PATCH: 8
PATCH: 9
PATCH: 10
PATCH: 11
PATCH: 12
PATCH: 13
PATCH: 14
PATCH: 15
PATCH: 17
PATCH: 18
PATCH: 19
PATCH: 20
PATCH: 21
PATCH: 22
PATCH: 23
PATCH: 24
PATCH: 25
PATCH: 26
PATCH: 27
PATCH: 28
PATCH: 29
PATCH: 30
PATCH: 31
PATCH: 32
PATCH: 33
PATCH: 34
PATCH: 35
PATCH: 36
PATCH: 37
PATCH: 38
PATCH: 39
PATCH: 40
PATCH: 41
PATCH: 42
PATCH: 43
PATCH: 44
PATCH: 45
PATCH: 46
PATCH: 47
PATCH: 48
PATCH: 49
PATCH: 50
PATCH: 51
PATCH: 52
PATCH: 53
PATCH: 54
PATCH: 55
PATCH: 56
PATCH: 57
PATCH: 58
PATCH: 59
PATCH: 60
PATCH: 61
PATCH: 62
PATCH: 63
PATCH: 64
PATCH: 65
PATCH: 66
PATCH: 67
PATCH: 68
PATCH: 69
PATCH: 70
PATCH: 71
PATCH: 72
PATCH: 73
PATCH: 74
PATCH: 75
PATCH: 76
PATCH: 77
PATCH: 78
PATCH: 79
PATCH: 80


---